<a href="https://colab.research.google.com/github/True-star-580/final_Multimodal-Sentiment-Analysis-with-MOSEI-Dataset/blob/main/inference_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#

In [6]:
!git clone https://github.com/True-star-580/final_Multimodal-Sentiment-Analysis-with-MOSEI-Dataset.git
%cd final_Multimodal-Sentiment-Analysis-with-MOSEI-Dataset

fatal: destination path 'final_Multimodal-Sentiment-Analysis-with-MOSEI-Dataset' already exists and is not an empty directory.
/content/final_Multimodal-Sentiment-Analysis-with-MOSEI-Dataset


In [7]:
!pip install -r requirements.txt -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 23.9 MB/s eta 0:00:00


In [8]:
!pip install -q langdetect

  Preparing metadata (setup.py) ... done


In [10]:
# you will be prompted through a dialog box to provide path for the model weights; was sent via email

In [41]:
%%writefile test_script.py
# inference_dual.py
import os
import sys
import logging
from pathlib import Path
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, pipeline
from moviepy.editor import VideoFileClip


sys.path.append(str(Path(__file__).resolve().parent))

# Import BOTH model architectures
from src.models.fusion import TransformerFusionModel as ModelV1
from src.models.fusion_v2 import MultilingualSentimentClassifier as ModelV2
from config import (
    TEXT_EMBEDDING_DIM, HIDDEN_DIM, NUM_ATTENTION_HEADS,
    NUM_TRANSFORMER_LAYERS, DROPOUT_RATE
)

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# --- Constants and Mappings ---
LABEL_INT_TO_STR_V2 = {0: 'SNEG', 1: 'WNEG', 2: 'NEUT', 3: 'WPOS', 4: 'SPOS'}

def score_to_class_v1(score):
    """Converts a V1 regression score to a V2 class string."""
    if -3.0 <= score <= -2.0: return 'SNEG'
    if -2.0 < score <= -0.5: return 'WNEG'
    if -0.5 < score < 0.5: return 'NEUT'
    if 0.5 <= score < 2.0: return 'WPOS'
    if 2.0 <= score <= 3.0: return 'SPOS'
    return 'NEUT' # Default case


def extract_audio_from_video(video_path, audio_output_path):
    try:
        video_clip = VideoFileClip(str(video_path))
        audio_clip = video_clip.audio
        if audio_clip is None: return False
        audio_clip.write_audiofile(str(audio_output_path), codec='pcm_s16le', logger=None)
        video_clip.close()
        return True
    except Exception: return False

def get_transcript_and_language(audio_path, device):
    """Generates transcript and detects language using Whisper."""
    try:
        pipe = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device)
        # The 'return_timestamps' flag can provide more metadata, including language
        result = pipe(str(audio_path), generate_kwargs={"language": None}, return_timestamps=False)
        transcript = result['text'].strip()

        # Whisper's pipeline doesn't directly expose language in the simple output,
        # but we can infer it. For simplicity, we'll check for Chinese characters.
        language = 'chinese' if any('\u4e00' <= char <= '\u9fff' for char in transcript) else 'english'

        logger.info(f"Detected Language: {language} | Transcript: '{transcript[:50]}...'")
        return transcript, language
    except Exception as e:
        logger.error(f"Failed to transcribe audio {audio_path}: {e}")
        return "", "unknown"

def get_text_embedding(text, tokenizer, bert_model, device):
    try:
        inputs = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        inputs = {key: val.to(device) for key, val in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs)
            embedding = outputs.last_hidden_state
        return embedding
    except Exception: return torch.zeros(1, 512, TEXT_EMBEDDING_DIM)

# --- Main Inference Logic ---

def run_dual_inference():
    """
    Main function to run hybrid inference using both V1 and V2 models.
    """
    # 1. Get user input
    model_v1_path_str = input("Enter path to the Multimodal Fusion Model (V1, .pt): ").strip()
    model_v2_path_str = input("Enter path to your FINETUNED Multilingual model (V2, .pt): ").strip()
    video_dir_str = input("Enter path to the directory with the .mp4 files: ").strip()

    model_v1_path = Path(model_v1_path_str)
    model_v2_path = Path(model_v2_path_str)
    video_dir = Path(video_dir_str)

    # Validate paths
    if not model_v1_path.exists() or not model_v2_path.exists() or not video_dir.is_dir():
        logger.error("One or more paths are invalid. Please check and try again.")
        return

    # 2. Setup Device and Feature Extractor
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    logger.info(f"Using device: {DEVICE}")

    logger.info("Loading Multilingual BERT for feature extraction...")
    bert_model_name = "bert-base-multilingual-cased"
    tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
    feature_extractor_bert = AutoModel.from_pretrained(bert_model_name).to(DEVICE)
    feature_extractor_bert.eval()

    # 3. Load Models
    logger.info("Loading Model V1 (Multimodal Fusion)...")
    model_v1 = ModelV1(
        text_dim=TEXT_EMBEDDING_DIM, audio_dim=0, visual_dim=0, hidden_dim=HIDDEN_DIM,
        num_layers=NUM_TRANSFORMER_LAYERS, num_heads=NUM_ATTENTION_HEADS, dropout_rate=DROPOUT_RATE
    )

    logger.warning("Skipping V1 model loading due to embedding mismatch. Proceeding with V2-only logic.")

    logger.info("Loading Model V2 (Finetuned Multilingual)...")
    # First, create the model structure with the base BERT
    model_v2_structure = ModelV2(feature_extractor_bert, hidden_dim=HIDDEN_DIM, num_classes=5)
    # Then, load the fine-tuned weights
    model_v2_structure.load_state_dict(torch.load(model_v2_path, map_location=DEVICE))
    model_v2 = model_v2_structure.to(DEVICE)
    model_v2.eval()
    logger.info("Models loaded successfully.")

    # 4. Process each video file
    video_files = list(video_dir.glob("*.mp4"))
    predictions_data = []
    temp_dir = Path("./temp_audio_dual")
    temp_dir.mkdir(exist_ok=True)

    for video_path in tqdm(video_files, desc="Processing Videos"):
        video_id = video_path.stem
        temp_audio_path = temp_dir / f"{video_id}.wav"

        if not extract_audio_from_video(video_path, temp_audio_path):
            continue

        transcript, language = get_transcript_and_language(temp_audio_path, DEVICE)
        if not transcript:
            os.remove(temp_audio_path)
            continue

        # Prepare input for V2 model (tokenizer is part of the V2 pipeline)
        inputs = tokenizer(transcript, padding=True, truncation=True, max_length=512, return_tensors="pt")
        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

        # Get V2 Prediction
        with torch.no_grad():
            logits = model_v2(**inputs)
            prediction_idx = torch.argmax(logits, dim=1).item()
            final_prediction_label = LABEL_INT_TO_STR_V2[prediction_idx]

        logger.info(f"Video: {video_path.name}, Language: {language}, Final Prediction: {final_prediction_label}")
        predictions_data.append({"ID": video_path.name, "Label": final_prediction_label})

        os.remove(temp_audio_path)

    # 5. Save results to CSV
    if predictions_data:
        df = pd.DataFrame(predictions_data)
        output_csv_path = video_dir / "predictions_dual.csv"
        df.to_csv(output_csv_path, index=False)
        logger.info(f"Inference complete! Predictions saved to {output_csv_path}")
        print(f"INFERENCE COMPLETE! Predictions saved to {output_csv_path}")

    # Clean up temp directory
    temp_dir.rmdir()


if __name__ == "__main__":
    run_dual_inference()

Overwriting test_script.py


In [16]:
!python test_script.py

2025-08-15 16:16:32.803231: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755274592.824677   50221 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755274592.831193   50221 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755274592.847784   50221 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755274592.847811   50221 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755274592.847817   50221 computation_placer.cc:177] computation placer alr